In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import numpy
import pandas
from IPython.display import display
import custom
from konlpy.tag import Kkma as word_div
from tqdm import tqdm
import threading

In [2]:
class Preprocessor() :
    def __init__(self) :
        self._result = None
        self._error = None
    def _process_with_timeout(self, s : str, timeout : int) :
        def work() :
            try :
                self._result = word_div().morphs(s)
            except Exception as e :
                self._error = e        
        self._error = None
        thread = threading.Thread(target=work, daemon=True)
        thread.start()
        thread.join(timeout=timeout)

        if thread.is_alive() :
            raise Exception(f"Error : Time Exceeded")
        if self._error is not None :
            raise Exception(str(self._error))

        return self._result
    def process(self, s : str, timeout : int = 10) :
        result = self._process_with_timeout(s, timeout)
        return result


In [3]:
print(Preprocessor().process("안녕하세요"))
# print(Preprocessor().process("진짜 조낸 재밌다 굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿"))

['안녕', '하', '세요']


In [4]:
original_df = pandas.read_csv("data/ratings_test.txt", sep = "\t", encoding = "UTF8")

display(original_df)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [5]:
df = original_df[['document','label']]

for i in tqdm(range(len(df))) :
    text = custom.text_preprocess_kor(str(df.iloc[i,0]), end_mark = True)
    if text == " " :
        df.iloc[i,0] = text
        continue
    try :
        df.iloc[i,0] = " ".join(Preprocessor().process(text))
    except Exception as e :
        df.iloc[i,0] = " "
        print(str(e), f"문장 : {text}")
    
word_len = []
for i in range(len(df)) :
    word_len.append(len(str(df.iloc[i,0]).split()))
df["word_len"] = word_len

df = df.drop(df[df["word_len"] == 0].index, axis = 0)

display(df)


 67%|█████████████████████████████████████████████████▌                        | 33452/50000 [15:09<1:58:49,  2.32it/s]

Error : Time Exceeded 문장 : 진짜 조낸 재밌다 굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿굿


 70%|███████████████████████████████████████████████████▌                      | 34802/50000 [16:03<2:51:53,  1.47it/s]

Error : Time Exceeded 문장 : 황시욬황시욬황시욬황시욬황시욬황시욬황시욬황시욬황시욬황시욬


 82%|████████████████████████████████████████████████████████████▊             | 41131/50000 [20:47<2:40:48,  1.09s/it]

Error : Time Exceeded 문장 : 우리 나라 드라마 계가 망해간다는것을 몸소 보여준다. 연기력 쓰레기에 그리 좋은 스토리도 아닌데 이렇게 호응해주는 사람들이 신기할 지경. 왕의 연애 소설이나 다룬 작품. 역사적 사실이 담겨있지도 않은 사극을 봐봤자 몰입도도 떨어지고 말도 안되는 설정.


100%|████████████████████████████████████████████████████████████████████████████| 50000/50000 [25:51<00:00, 32.24it/s]


,document,label,word_len
0,굳,1,1
2,뭐 야 이 평점 들 은 . 나쁘 지 는 않 지만 점 짜리 는 더더욱 아니 잖아,0,18
3,지루 하 지 는 않 은데 완전 막장 임 . 도 ㄴ 주고 보 기에 는 .,0,17
4,만 아니 었 어도 별 다섯 개 주 었 을 터 이 ㄴ데 . 왜 로 나오 아서 저 의 ...,0,28
5,음악 이 주가 되 ㄴ 최고 의 음악 영화,1,9
...,...,...,...
49995,오랜만 에 평점 로 긴하 었 네 킹 왕 짱 쌈 뽕 하 ㄴ 영화 를 만나 었 습니다 ...,1,26
49996,의지 박약 들 이나 하 는 거 이 다 탈영 은 일단 주인공 김 대희 닮 았 고 이등...,0,22
49997,그림 도 좋 고 완성도 도 높 았 지만 . 보 는 내내 불안 하 게 만들 ㄴ다,0,18
49998,절대 보 아서 는 안 되 ㄹ 영화 . 재미 도 없 고 기분 만 잡치 고 . 하 ㄴ ...,0,25


In [6]:

df.to_csv("data/korean_movie_test.txt", sep = "\t", encoding = "UTF8")

In [7]:
print(df["word_len"].max())
print(df["word_len"].mean())
print(df["word_len"].std())

105
19.236804449908302
16.350658068148533
